AMI and ICSI Json Loads

In [1]:
import json
import os
import pandas as pd
import statistics
#from unsupervised_topic_segmentation import core, eval, types, dataset
import create_test_data

import re #remove later

ModuleNotFoundError: No module named 'create_test_data'

In [2]:
%ls
#%cd ..

ami                          data_for_s3.pickle          lambda_function.py
ami.pickle                   data_to_s3.py               lambda_function_old.py
ami_isci_json_loading.ipynb  embed_lambda/               testing.ipynb
ami_pickle                   embed_lambda.zip            testing_aws.ipynb
aws_instructions.txt         environment_op.yml
batch_function.py            import_torch_for_lambda.py
/home/opinney/final_ml/decoding-democracy


# Saving data for self labeling 

In [6]:
t_cd = create_test_data.transcript_pickle_to_df()

naive method: split on sentences containing agenda or file number
note that some sentences are artificially long, so we should not be splitting ami and isci on sentences either

In [3]:
#eg of long sentence
# So yeah let me answer that question two ways first at the kind of macro level which I think you made a good statement there about we you know if we're going to be seeing the city grow and achieving some of the mayor's population goals we'll need to add housing units both where people want to live but across the city so I think that's a fair thing that we want to consider as we're carrying out this project in my opening comments I also said we're not bringing you final ideas or final final recommendations today so as far as like some of the very site-specific things you got into I don't pretend to come to you and say those because those are uniquely desirable residential neighborhoods because of their low density well and the thing I was going to mention the second point you know you you did you almost set us up too well to now make a plug that you know you talked about the Bayview neighborhood specifically and that is an area where we are doing detailed area planning right now led by Alderwoman Dimitrovich just tonight at from between 5 and 7 p.m. we have a meeting at South Shore Pavilion where we're going to present some initial recommendations for Bayview that have come out of the the planning process there so we are certainly looking at that and you know I've been we've certainly heard recommendations from residents in Bayview and I'm not going to pretend this has been unanimous but there's been a lot of people who said we do want to see growth in Bayview and we're certainly trying to accommodate that and and use that specific neighborhood planning process to plan for it so I think we're continuing to do the things we've always done which is plan you know both do neighborhood planning but also acknowledge the thing that you said at the start which is some of these issues when you look at regulatory tools and growth policy we do probably have to do some level of citywide analysis there so that's why we're proposing to undertake this project if you're right that depending on the way that this project unfolds like we certainly want input from the council and that's why we're proposing is a two-phase project where at the end of phase one we come back to you all at the council I presume that would come to ZND here and before we start actually writing you know writing code updates we say here are the general findings that we found here is the some general recommendations around things we want to that we want to advance and I think we'll ask the council this committee and then the council to go on record supporting that before we start right you said you know getting in the weeds of writing code because we don't want to bring you something that's not going to be able to get the support of this council additional questions Mr. Chair all the way yes please go ahead yes yes good afternoon Mr. Chair thank you and thanks for all the love for Bayview we love it and good looking out we are as Sam pointed out in almost a year and a half a very intense planning effort because we want to be part of the city solution we want to be in the driver's seat of our own future and we know we've had a lot of changes and so we are really working on a forward thinking vision how we can be more inclusive what that means what that looks and feels like and what changes need to happen and we've had some really interesting conversations in in Bayview that I'm happy to be part of that I don't think it's taken place at this level for a while however the question I had was kind of how you know how does this complement compare because I wouldn't want to support a citywide plan today that includes changes that I haven't finalized within my own district plan and I just wanted to understand that and we do have that meeting tonight where we're actually it's an open house we're going to first respond to the first draft that's that's what I'm interested in is I do believe we need a citywide plan but I mean where there's districts that are going through this intensive process I want to make sure that that's complemented and not contradicting Mr. Chair.


In [4]:
import random
random.randint(0,412) #will look at transcript 42
#text=t_cd.iloc[42]['sentences']
#df_temp=pd.DataFrame({'sentences':text})
#df_temp.to_csv("transcript_x.csv")


97

# ICSI
Not using icsi at the moment

In [5]:
#not using icsi at the moment
icsi_path='ami-and-icsi-corpora-master/icsi-corpus/output/dialogueActs/'

text_jsons_isci=[]

file_paths = os.listdir(icsi_path)
file_paths = list(filter(lambda x: x.endswith(".json"),file_paths))

for file_path in file_paths:
    #probably can comment out the text section because text is contained in topic 
    with open( icsi_path + file_path ) as file:
        json_load = json.loads(file.read())
    text_jsons_isci.append(json_load)
    
print(len(text_jsons_isci))


75


In [6]:
text_lengths_isci=[len(x) for x in text_jsons_isci]
statistics.median(text_lengths_isci)


1454

isci has fewer transcripts (75 versus 136), but might be better for our purposes because transcripts are longer (1454 utterances precleaning, rather than 251 in ami postcleaning), 

# AMI
Using AMI instead

In [3]:
#ami loading
text_path='data/ami-and-icsi-corpora-master/ami-corpus/output/dialogueActs/'
topic_path='data/ami-and-icsi-corpora-master/ami-corpus/output/topics/'

text_jsons=[]
topic_jsons=[]

file_paths = os.listdir(topic_path)
file_paths = list(filter(lambda x: x.endswith(".json"),file_paths))

for file_path in file_paths:
    #probably can comment out the text section because text is contained in topic 
    with open( text_path + file_path ) as file:
        json_load = json.loads(file.read())
    text_jsons.append(json_load)
    
    with open( topic_path + file_path ) as file:
        json_load = json.loads(file.read())
    topic_jsons.append(json_load)

print(len(text_jsons)==len(topic_jsons))
len(topic_jsons)


True


136

In [4]:
#ami loading
FILLERS=["um", "uh", "oh", "hmm", "mm-hmm", "uh-uh", "you know"]
topic_path='data/ami-and-icsi-corpora-master/ami-corpus/output/topics/'

def jsons_to_dict_list(topic_path):

    dict_list=[]

    file_paths = os.listdir(topic_path)
    file_paths = list(filter(lambda x: x.endswith(".json"),file_paths))

    for file_path in file_paths:    
        with open( topic_path + file_path ) as file:
            json_load = json.loads(file.read())
        dict_list.append(json_load)

    return dict_list

#put list of jsons into df
topic_jsons=jsons_to_dict_list(topic_path)

ami_ids=["AMI_"+str(x) for x in range(len(topic_jsons))]

t_ami = pd.DataFrame({'transcript_id':ami_ids,
                      'topic_json':topic_jsons})

def clean_topic_json(topic_json,fillers):

    
    df_temp=pd.DataFrame()
    has_topic_desc=len(set([x['topic'] for x in topic_json]))>1

    for index, topic in enumerate(topic_json): 
        text = [x['text'] for x in topic['dialogueacts']]
        if df_temp.empty:
            df_temp=pd.DataFrame({'sentences':text,'topic_count':index,'topic_desc':topic['topic'],'has_topic_desc':has_topic_desc})
        else:
            df_temp=pd.concat([df_temp,pd.DataFrame({'sentences':text,'topic_count':index,'topic_desc':topic['topic'],'has_topic_desc':has_topic_desc})])    

    df_clean=dataset.preprocessing(df_temp,'sentences',FILLERS.copy(),min_caption_len=20)
    
    #text_length=len(df_clean)
    #topic_length=df_clean.groupby(['topic_count']).size().mean()
    #transcript=None

    return df_clean

    
    text_lengths.append()
    topic_lengths.append()
    transcripts.append(df_clean)
    
ami_ids=["AMI_"+str(x) for x in range(len(transcripts))]
sentences=[list(x['sentences']) for x in transcripts]
topic_counts=[list(x['topic_count']) for x in transcripts]

t_ami = pd.DataFrame({'transcript_id':ami_ids,
    'sentences':sentences,
    'topic_counts':topic_counts,
    'length':text_lengths,
    'avg_topic_length':topic_lengths
    })

t_ami

NameError: name 'transcripts' is not defined

# Viewing AMI

Notice that the text has a lot of filler. For example: <vocalsound>, um, uh, mm-hmm, uh-uh, oh. We want to keep in <gap> and <disfmarker>

We can see that the sentence structure is different than our council text. There are many shorter sentences, some utterances are interupted and others contain multiple sentences. 
    

In [8]:
text = [x['text'] for x in text_jsons[0]]
text_clean = dataset.preprocessing(pd.DataFrame({'sentences':text}),'sentences')
text_clean

,index,sentences
0,14,Now I have my screen back too
1,20,we have presentations
2,21,"So first , it's your turn"
3,35,let's talk about the interface concept
4,36,", first I'll I'll discuss the buttons we ju..."
...,...,...
329,769,It has to be ready in the next meeting
330,774,Next meeting is called the detailed design
331,776,Everyth everything has to be ready
332,778,Thanks for your attention


Topics 

In [9]:
print(topic_jsons[0][0].keys())
print(set([x['topic'] for x in topic_jsons[0]])) 

dict_keys(['id', 'topic', 'description', 'dialogueacts', 'subtopics'])
{'opening', 'marketing expert presentation', 'industrial designer presentation', 'closing', 'discussion', 'interface specialist presentation'}


# Clean AMI

This code takes 3.7 seconds to run. Make sure not to delete the .copy()

# Drew, do you understand what this code does?

In [10]:
PARALLEL_INFERENCE_INSTANCES = 20

def embed_bert(df, caption_col_name, parallel_inference_instances = PARALLEL_INFERENCE_INSTANCES):
    '''copied from core.topic_segmentation_bert()'''
    # parallel inference
    batches_features = []
    for batch_sentences in core.split_list(
        df[caption_col_name], parallel_inference_instances
    ): # splits into sequential batches such that total number of batches equals INSTANCES value
        batches_features.append(core.get_features_from_sentence(batch_sentences)) # list of tensors of size (1,768), one for each sentence
    features = core.flatten_features(batches_features)   # changes back to list of length 768 tensors, one for each sentence in dataset
    return features


In [11]:
test_data = pd.DataFrame(data={'meeting_id':t_cd.iloc[0]['transcript_id'],'caption':t_cd.iloc[0]['sentences']})

hold=embed_bert(test_data,'caption',20)
len(hold)

459

In [14]:
df_temp

NameError: name 'df_temp' is not defined

In [15]:
transcripts=[]
text_lengths=[]
topic_lengths=[] #can remove later
FILLERS=["um", "uh", "oh", "hmm", "mm-hmm", "uh-uh", "you know"]

for topic_json in topic_jsons:

    df_temp=pd.DataFrame()
    has_topic_desc=len(set([x['topic'] for x in topic_json]))>1

    for index, topic in enumerate(topic_json): 
        text = [x['text'] for x in topic['dialogueacts']]
        if df_temp.empty:
            df_temp=pd.DataFrame({'sentences':text,'topic_count':index,'topic_desc':topic['topic'],'has_topic_desc':has_topic_desc})
        else:
            df_temp=pd.concat([df_temp,pd.DataFrame({'sentences':text,'topic_count':index,'topic_desc':topic['topic'],'has_topic_desc':has_topic_desc})])    

    df_clean=dataset.preprocessing(df_temp,'sentences',FILLERS.copy(),min_caption_len=20)
    
    
    text_lengths.append(len(df_clean))
    topic_lengths.append(df_clean.groupby(['topic_count']).size().mean())
    transcripts.append(df_clean)
    
df_clean

,index,sentences,topic_count,topic_desc,has_topic_desc
0,2,This is our first team meeting,0,None,False
1,3,"I'll be your Project Manager for today , for t...",0,None,False
2,5,will be giving this presentation for you to k...,0,None,False
3,7,that's the agenda for today,0,None,False
4,9,"of course we're new to each other ,",0,None,False
...,...,...,...,...,...
187,1,the the personal coach will give you the your ...,3,None,False
188,2,So we'll just meet back in here thirty minutes,3,None,False
189,3,I'm sure we have that,3,None,False
190,9,thanks for attending,3,None,False


calculate average topic length

In [12]:
sum(text_lengths)/len(text_lengths) #this will be an issue because our texts are longer in our city council transcripts

216.28676470588235

In [13]:
import statistics
statistics.median(topic_lengths) #this is median of averages, not total average

24.944444444444443

## Make AMI Transcripts match Council Data format

In [14]:
ami_ids=["AMI_"+str(x) for x in range(len(transcripts))]
sentences=[list(x['sentences']) for x in transcripts]
topic_counts=[list(x['topic_count']) for x in transcripts]

In [15]:
t_cd = create_test_data.transcript_pickle_to_df()
t_ami = pd.DataFrame({'transcript_id':ami_ids,
    'sentences':sentences,
    'topic_counts':topic_counts,
    'length':text_lengths,
    'avg_topic_length':topic_lengths
    })

t_cd

,transcript_id,sentences
0,d0a7e5864959,"[And older woman Jocasta Zamarripa., Shortly, ..."
1,e9a7a8ac9081,"[Meeting., My name is Cavalier Johnson., I'm c..."
2,694b0e5b01a7,[Joining you this morning is Vice Chair Alderm...
3,ad734a167e5a,"[Our first meeting of 2020, the Judiciary and ..."
4,fe845b99f32e,"[Alderman Hamilton., Here., Kovach., Here., Ba..."
...,...,...
407,468bb3242311,[Commission for Tuesday September 1st 2020 at ...
408,16327f867c7e,[Our 2021 meeting of the Fire and Police Commi...
409,ea6416ba848c,"[Thank you, Mr. President., You know, my colle..."
410,d32da631854f,"[This meeting will come to order, this council..."


In [16]:
t_ami

,transcript_id,sentences,topic_counts,length,avg_topic_length
0,AMI_0,"[Now I have my screen back too , we have pre...","[0, 0, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, ...",230,38.333333
1,AMI_1,"[This is our third meeting already , I hope yo...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...",78,11.142857
2,AMI_2,"[Everybody found his place again ? Yeah ?, thi...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...",90,12.857143
3,AMI_3,"[Good morning everybody , I'm glad you could ...","[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 2, 2, 2, ...",135,16.875000
4,AMI_4,"[English from now on , Where are are all the ...","[0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, ...",251,22.818182
...,...,...,...,...,...
131,AMI_131,"[start of the first meeting , Right , so agend...","[0, 0, 0, 0, 0, 1, 1, 1, 1, 1, 1, 2, 2, 2, 2, ...",180,20.000000
132,AMI_132,"[I wanna find our if our remote works , here...","[0, 0, 0, 0, 0, 0, 0, 1, 1, 1, 1, 1, 1, 2, 2, ...",232,38.666667
133,AMI_133,[So do we need to re-train Mike on how to put ...,"[0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 1, 1, 1, 1, 1, ...",84,12.000000
134,AMI_134,"[That's as far as it goes , good morning eve...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...",152,38.000000


In [17]:
hold=create_test_data.generate_segment(t_ami, doc_count_limit = 5, sentence_min = 20, supervised = True)  

In [18]:
print(hold[3])
print(set(hold[2]))

1
{0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11}


# Embed Data

In [ ]:
from unsupervised_topic_segmentation import core, eval, types, dataset
import create_test_data
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import pickle

In [24]:
t_cd #instead of transcripts



,transcript_id,sentences
0,d0a7e5864959,"[And older woman Jocasta Zamarripa., Shortly, ..."
1,e9a7a8ac9081,"[Meeting., My name is Cavalier Johnson., I'm c..."
2,694b0e5b01a7,[Joining you this morning is Vice Chair Alderm...
3,ad734a167e5a,"[Our first meeting of 2020, the Judiciary and ..."
4,fe845b99f32e,"[Alderman Hamilton., Here., Kovach., Here., Ba..."
...,...,...
407,468bb3242311,[Commission for Tuesday September 1st 2020 at ...
408,16327f867c7e,[Our 2021 meeting of the Fire and Police Commi...
409,ea6416ba848c,"[Thank you, Mr. President., You know, my colle..."
410,d32da631854f,"[This meeting will come to order, this council..."


In [ ]:
new_algorithm = types.BERTSegmentation(
    sentence_comparison_window=50,
    text_tiling=types.NewSegmentation(
        stdevs=1))

new_algorithm

In [27]:
test_data = pd.DataFrame(data={'meeting_id':t_cd.iloc[0]['transcript_id'],'caption':t_cd.iloc[0]['sentences']})

#test_data = pd.DataFrame(data={'meeting_id':'d0a7e5864959','caption':t_cd['d0a7e5864959']})
test_data = dataset.add_durations(test_data)  # "duration" column is just number of words
test_data = dataset.preprocessing(test_data,min_caption_len=5)  # note that this adds (old) `index` column, but topic_segmentation uses actual index
test_data

,index,meeting_id,start_time,end_time,caption
0,0,d0a7e5864959,0,5,And older woman Jocasta Zamarripa
1,1,d0a7e5864959,5,15,"Shortly, Alderman Rainey and Alderman Hamilton..."
2,2,d0a7e5864959,15,19,So let's get started
3,3,d0a7e5864959,19,36,"Item nber 211814, reappointment of Brian Rott ..."
4,4,d0a7e5864959,36,42,"Brian Rott, I see you"
...,...,...,...,...,...
483,454,d0a7e5864959,4342,4349,And that concludes our committee meeting today
484,455,d0a7e5864959,4349,4351,Thank you
485,456,d0a7e5864959,4351,4357,See you all in the community
486,457,d0a7e5864959,4357,4360,Thanks to additionally


Rerun the core code

In [35]:
df = test_data
meeting_id_col_name = 'meeting_id'
start_col_name = 'start_time'
end_col_name = 'end_time'
caption_col_name = 'caption'

new_algorithm = types.BERTSegmentation(
    sentence_comparison_window=50,
    text_tiling=types.NewSegmentation(
        stdevs=1))

topic_segmentation_configs = new_algorithm



In [37]:
PARALLEL_INFERENCE_INSTANCES = 20

# parallel inference
batches_features = []
for batch_sentences in core.split_list(
    df[caption_col_name], PARALLEL_INFERENCE_INSTANCES
): # splits into sequential batches such that total number of batches equals INSTANCES value
    batches_features.append(core.get_features_from_sentence(batch_sentences)) # list of tensors of size (1,768), one for each sentence
features = core.flatten_features(batches_features)   # changes back to list of length 768 tensors, one for each sentence in dataset


In [44]:
len(features)
len(test_data)

features[0].shape

torch.Size([1, 768])

In [46]:
len(batches_features)

20